In [4]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense, GRU, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.utils import plot_model
import numpy as np
import json

Using TensorFlow backend.


In [7]:
train_dir_name = '/Users/KevinChuang/PycharmProjects/Screenshot-to-code-in-Keras/local/Bootstrap/train/'
test_dir_name = '/Users/KevinChuang/PycharmProjects/Screenshot-to-code-in-Keras/local/Bootstrap/eval/'
# Read a file and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []
    # Load all the files and order them
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "npz":
            # Load the images already prepared in arrays
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

train_features, train_texts = load_data(train_dir_name)
test_features, test_texts = load_data(test_dir_name)

In [9]:
# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
tokenizer.fit_on_texts([load_doc('/Users/KevinChuang/PycharmProjects/Screenshot-to-code-in-Keras/local/Bootstrap/resources/bootstrap.vocab')])

# summarize what was learned
print('Tokenizer word counts: ', tokenizer.word_counts)
print('Tokenizer document count: ', tokenizer.document_count)
print('Tokenizer word index: ', tokenizer.word_index)
print('Tokenizer word docs: ', tokenizer.word_docs)

Tokenizer word counts:  OrderedDict([(',', 1), ('{', 1), ('}', 1), ('small-title', 1), ('text', 1), ('quadruple', 1), ('row', 1), ('btn-inactive', 1), ('btn-orange', 1), ('btn-green', 1), ('btn-red', 1), ('double', 1), ('<START>', 1), ('header', 1), ('btn-active', 1), ('<END>', 1), ('single', 1)])
Tokenizer document count:  1
Tokenizer word index:  {',': 1, '{': 2, '}': 3, 'small-title': 4, 'text': 5, 'quadruple': 6, 'row': 7, 'btn-inactive': 8, 'btn-orange': 9, 'btn-green': 10, 'btn-red': 11, 'double': 12, '<START>': 13, 'header': 14, 'btn-active': 15, '<END>': 16, 'single': 17}
Tokenizer word docs:  {'btn-orange': 1, '<END>': 1, 'header': 1, 'single': 1, 'btn-red': 1, '<START>': 1, 'quadruple': 1, '{': 1, 'text': 1, ',': 1, 'row': 1, 'small-title': 1, 'btn-active': 1, 'btn-inactive': 1, 'double': 1, '}': 1, 'btn-green': 1}


In [10]:
# # Add one spot for the empty word in the vocabulary 
vocab_size = len(tokenizer.word_index) + 1
max_length = 48
print(train_texts[0])
print(tokenizer.texts_to_sequences([train_texts[0]]))

<START> header { btn-inactive , btn-inactive , btn-inactive , btn-active } row { single { small-title , text , btn-red } } row { quadruple { small-title , text , btn-green } quadruple { small-title , text , btn-orange } quadruple { small-title , text , btn-red } quadruple { small-title , text , btn-orange } } row { double { small-title , text , btn-red } double { small-title , text , btn-orange } } <END>
[[13, 14, 2, 8, 1, 8, 1, 8, 1, 15, 3, 7, 2, 17, 2, 4, 1, 5, 1, 11, 3, 3, 7, 2, 6, 2, 4, 1, 5, 1, 10, 3, 6, 2, 4, 1, 5, 1, 9, 3, 6, 2, 4, 1, 5, 1, 11, 3, 6, 2, 4, 1, 5, 1, 9, 3, 3, 7, 2, 12, 2, 4, 1, 5, 1, 11, 3, 12, 2, 4, 1, 5, 1, 9, 3, 3, 16]]


77

In [11]:
def preprocess_data(texts, features, max_sequence):
    X, y, image_data = list(), list(), list()
    sequences = tokenizer.texts_to_sequences(texts)
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(image_data), np.array(X), np.array(y)

# image_data, x_sequence, labels = preprocess_data([train_texts[0]], [train_features[0]], 150)

In [12]:
# data generator, intended to be used in a call to model.fit_generator()
# This will save memory and prevent MemoryErrors from numpy
def data_generator(descriptions, features, n_step, max_sequence):
    '''n_steps = batch_size'''
    # loop until we finish training
#     print('Data Generator initialized...')
    while 1:
        # loop over photo identifiers in the dataset
        for i in range(0, len(descriptions), n_step):
            batch = 0
            Ximages, XSeq, y = list(), list(),list()
            for j in range(i, min(len(descriptions), i+n_step)):
                image = features[j]
                # retrieve text input
                desc = descriptions[j]
                # generate input-output pairs
                in_img, in_seq, out_word = preprocess_data([desc], [image], max_sequence)
                for k in range(len(in_img)):
                    Ximages.append(in_img[k])
                    XSeq.append(in_seq[k])
                    y.append(out_word[k])
            # yield this batch of samples to the model
#             print('Yielding batch %d \n' % i)
            yield [[np.array(Ximages), np.array(XSeq)], np.array(y)]

In [14]:
generator = data_generator(train_texts, train_features, 1, 150)
x, y =next(generator)
print(x[0].shape)
print(x[1].shape)
print(y.shape)

(76, 256, 256, 3)
(76, 48)
(76, 18)


In [ ]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(train_features)

In [ ]:
# Parameters
params = {'dim': (256, 256),
          'batch_size': 1,
          'n_classes': 18,
          'n_channels': 1,
          'shuffle': True}

partition = {'train': [i for i in os.listdir(train_dir_name) if i[-3:] == '.npz'],
             'validation': [v for v in os.listdir(test_dir_name) if v[-3:] == '.npz']}
labels = {i: i % 10 for i in range(n_samples+100)}

In [ ]:
class DataGenerator(keras.utils.Sequence):

    # initialization
    def __init__(self, dim_x = 256, dim_y = 256, batch_size = 32, number_features = 48, shuffle = True):

        self.dim_x = dim_x
        self.dim_y = dim_y
        self.batch_size = batch_size
        self.number_features = number_features
        self.shuffle = shuffle

    def __len__(self):
      'Denotes the number of batches per epoch'
      return int(np.floor(len(self.list_IDs) / self.batch_size))
        
    # randomize the indices of the list_IDs
    def __get_exploration_order(self, list_IDs):

        indices = np.arange(len(list_IDs))
        if self.shuffle == True:
            np.random.shuffle(indices)
        return indices

    # import the batch based on IDs
    def __data_generation(self, labels, list_IDs_temp):
        # initialization
        X1 = np.empty((self.batch_size, self.dim_x, self.dim_y, 3))
        X2 = np.empty((self.batch_size, self.number_features))
        Y = np.empty((self.batch_size), dtype = int)

        # generate data
        for i, ID in enumerate(list_IDs_temp):
            # import picture, resize and convert to array
            img = Image.open(paths[ID])
            arr = np.array(img)
            img_resized = imresize(arr, (self.dim_x, self.dim_y))

            # store picture
            X1[i, :, :, :] = img_resized / 255

            # get 'non-picture' features
            X2[i] = features[ID]

            # store class
            Y[i] = labels[ID]

        return X1, X2, to_categorical([Y], num_classes=18)[0]

    # an infinite loop that goes through the dataset and outputs one batch at a time
    def generate(self, labels, list_IDs):

        # infinite loop
        while 1:

            # generate order of exploration of dataset
            indices = self.__get_exploration_order(list_IDs)

            # generate batches
            imax = int(len(indices) / self.batch_size)

            for i in range(imax):
            # find list of IDs
            list_IDs_temp = [list_IDs[k] for k in indices[i * self.batch_size: (i + 1) * self.batch_size]]

            # generate data
            X1, X2, Y = self.__data_generation(labels, list_IDs_temp)

            yield [X1, X2], Y

In [ ]:
# # For convolution layers, remember that the output shape is: [(W-F)/S +1, (W-F)/S +1], 
# # where W=H= 256 and filter shape F=3, and stride S=1 or 2

# # Create the encoder
image_model = Sequential()
image_model.add(Conv2D(16, (3,3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
image_model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(256, 256, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = GRU(128, return_sequences=True)(language_model)
language_model = GRU(128, return_sequences=True)(language_model)

#Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = GRU(512, return_sequences=True)(decoder)
decoder = GRU(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
# optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
optimizer = Adam(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# plot_model(model, to_file='model.png')

# Serialize Model to JSON
# model_json = model.to_json()
# with open('GRU_model.json', 'w') as json_file:
#     json_file.write(model_json)

In [9]:
for l in image_model.layers:
    print('For %s,' % l)
    print('\tinput_shape: ', l.input_shape)
    print('\toutput_shape: ', l.output_shape)

For <keras.layers.convolutional.Conv2D object at 0x1113d07f0>,
	input_shape:  (None, 256, 256, 3)
	output_shape:  (None, 252, 252, 32)
For <keras.layers.convolutional.Conv2D object at 0x1113d0908>,
	input_shape:  (None, 252, 252, 32)
	output_shape:  (None, 252, 252, 32)
For <keras.layers.pooling.MaxPooling2D object at 0x10775e7f0>,
	input_shape:  (None, 252, 252, 32)
	output_shape:  (None, 126, 126, 32)
For <keras.layers.core.Dropout object at 0x1113d0e10>,
	input_shape:  (None, 126, 126, 32)
	output_shape:  (None, 126, 126, 32)
For <keras.layers.convolutional.Conv2D object at 0x1077799b0>,
	input_shape:  (None, 126, 126, 32)
	output_shape:  (None, 126, 126, 64)
For <keras.layers.convolutional.Conv2D object at 0x1113d0b70>,
	input_shape:  (None, 126, 126, 64)
	output_shape:  (None, 126, 126, 64)
For <keras.layers.pooling.MaxPooling2D object at 0x107dd0f28>,
	input_shape:  (None, 126, 126, 64)
	output_shape:  (None, 63, 63, 64)
For <keras.layers.core.Dropout object at 0x10775ee80>,
	inp

In [10]:
for l in model.layers:
    print('For %s,' % l)
    print('\tinput_shape: ', l.input_shape)
    print('\toutput_shape: ', l.output_shape)

For <keras.engine.input_layer.InputLayer object at 0x111506160>,
	input_shape:  (None, 48)
	output_shape:  (None, 48)
For <keras.layers.embeddings.Embedding object at 0x111506588>,
	input_shape:  (None, 48)
	output_shape:  (None, 48, 50)
For <keras.engine.input_layer.InputLayer object at 0x1114aada0>,
	input_shape:  (None, 256, 256, 3)
	output_shape:  (None, 256, 256, 3)
For <keras.layers.recurrent.GRU object at 0x1114f4780>,
	input_shape:  (None, 48, 50)
	output_shape:  (None, 48, 128)
For <keras.engine.sequential.Sequential object at 0x1113d0828>,
	input_shape:  (None, 256, 256, 3)
	output_shape:  (None, 48, 1024)
For <keras.layers.recurrent.GRU object at 0x111522ba8>,
	input_shape:  (None, 48, 128)
	output_shape:  (None, 48, 128)
For <keras.layers.merge.Concatenate object at 0x111678f60>,
	input_shape:  [(None, 48, 1024), (None, 48, 128)]
	output_shape:  (None, 48, 1152)
For <keras.layers.recurrent.GRU object at 0x111a4eda0>,
	input_shape:  (None, 48, 1152)
	output_shape:  (None, 48

In [11]:
# filepath= "weights/weights.epoch-{epoch:02d}--loss-{loss:.4f}--acc-{acc:.4f}--val_loss-{val_loss:.4f}--val_acc-{val_acc:.4f}.hdf5"
filepath="weights/weights.epoch-{epoch:02d}--loss-{loss:.4f}--acc-{acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath,verbose=1, 
                             save_weights_only=True, period=2)

# Create a callback that streams epoch results to a csv file.
csv_file = 'weights/training.log'
csv_logger = CSVLogger(csv_file)


callbacks_list = [checkpoint, csv_logger]

In [12]:
# Let's define the neural network hyperparameters here as a dict
# steps_per_epoch = # of training examples / batch_size
# steps_per_epoch = number of batches and number of times the weights are updated (backpropagation occurs to update weights)
config = {}
config['steps_per_epoch'] = 1500
config['epochs'] = 50
config['batch_size'] = 1500/config['steps_per_epoch']
config['validation_steps'] = 250
config_path = 'weights/config.json'

with open(config_path, 'w') as fh:
    json.dump(config, fh, indent=4, sort_keys=True)

In [ ]:
# for e in range(config['epochs']):
# #     print("epoch %d" % e)
#     for X_train, Y_train in data_generator(train_texts, train_features, 2, 150): # these comes in batches of 2 images
#         model.fit(X_train, Y_train, steps_per_epoch=750)

In [13]:
model_history = model.fit_generator(data_generator(train_texts, train_features, 1, 150),                             
                                    steps_per_epoch=config['steps_per_epoch'], 
                                    epochs=config['epochs'], 
                                    callbacks=callbacks_list, 
                                    verbose=1,
                                    workers=5)

Epoch 1/50
   1/1500 [..............................] - ETA: 16:15:30 - loss: 2.8978 - acc: 0.0395

KeyboardInterrupt: 

In [ ]:
# # Train and fit the model using a generator (more memory efficient)
# # steps_per_epoch = # of training examples / batch_size
# n_steps = batch
# model_history = model.fit_generator(data_generator(train_texts, train_features, 1, 150),                             
#                                     steps_per_epoch=config['steps_per_epoch'], 
#                                     epochs=config['epochs'], 
#                                     callbacks=callbacks_list, 
#                                     verbose=1,
#                                     workers=5,
#                                     validation_data=data_generator(test_texts, test_features, 1, 150),
#                                    validation_steps=config['validation_steps'])

In [ ]:
# model_history = model.fit_generator(data_generator(train_texts, train_features, 1, 150),                             
#                                     steps_per_epoch=config['steps_per_epoch'], 
#                                     epochs=config['epochs'], 
#                                     callbacks=callbacks_list, 
#                                     verbose=1,
#                                     use_multiprocessing=True,
#                                     validation_data=data_generator(test_texts, test_features, 1, 150),
#                                    validation_steps=config['validation_steps'])

In [ ]:
# Open the file
with open('K:\\Projects\\Autogeneration_of_Code_from_Images\\weights\\report.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model_history.model.summary(print_fn=lambda x: fh.write(x + '\n'))